# Case Study 3 - Synthetic Laccolith
- This example represent a laccolith, and it is built with 6 contact points: 3 contact points from the laccolith's roof and 3 contact points form the laccolith's floor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loop library
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer 
# from LoopStructural.modelling.intrusions import ellipse_function, parallelepiped_function

In [ ]:
from datetime import datetime

In [ ]:
lower_extent = [0,0,0]
upper_extent = [1,1,1]

## 1. Load data

### Input DataFrame description
> feature_name = name of the geological feature to be modelled. 

> (X, Y, Z) = data points location

> coord = structural frame coordinate

> val = values of the scalar field for interpolation of geological features

> (gx, gy, gz) = gradients of structural frame scalar fields

> intrusion_contact_type = roof/top or floor/base

> intrusion_side = TRUE if lateral contact, blank if not


In [ ]:
model_data = pd.read_csv('Synthetic Laccolith.csv')
intrusion_data = model_data[model_data['feature_name'] == 'laccolith'].copy()
model_data.head()

## 2. Geological model

### The ``create_and_add_intrusion`` function of the ``GeologicalModel`` class creates an ``IntrusionFeature`` in two steps: 

### Step 1 - Create the intrusion frame using the ``IntrusionFrameBuilder`` class:

IntrusionFrameBuilder creates the intrusion structural frame.
This object is a curvilinear coordinate system of the intrusion body, and it has three coordinates:
> coordinate 0: it represents an approximate location of the roof or floor contact of the intrusion, and its gradient representes the growth or inflation of the magma. It is constrained using data from either the roof or floor contact. If no inflation data is available, it is assumed that the gradient of this coordinate is perpendicular to the stratigraphy.

> coordinate 1: it represents the flow direction of the intrusion. Its gradient is parallel to the propagation of the magma, which can be constrained with flow direction measurements (e.g., AMS data). In this example, we only have contact data point, so we assume the propagation was parallel to the long axis of the intrusion. 

> coordinate 2: it is parallel to the long axis of the intrusion. 

The extent of the intrusion is defined as distances along the axes of the intrusion frame.

In [ ]:
intrusion_frame_data = model_data[model_data['feature_name'] == 'laccolith_frame'].copy()
intrusion_frame_data

### Step 2 - Compute threshold distances to constrain the intrusion extent, using the ``IntrusionBuilder`` class:

The IntrusionBuilder computes thresholds distances (along the structural frame coordinates) to constrain the intrusion lateral and vertical extent.

Distances along the axis of coordinate 2 are computed to constraint the intrusion lateral extent, and distances along the axis of coordinate 0 are computed to constained its vertical extent. 

Thresholds values are restricted using conceptual geometrical models representing the expected shape of the intrusion.


#### Conceptual model function to constraint lateral and vertical extent

These conceptual model are used to define the geometry of the intrusion contact where there is no data available.

They allow the modeler to integrate their interpretation of the intrusion geometry.

In this example, there are no lateral contact data, so the lateral extent of the lacolith is completely controlled by the lateral conceptual model. In this case we use an *ellipsis* to constrain it lateral extent. The roof of the laccolith is constrained using a bell-shaped curve. 

In [ ]:
# Conceptual models

def ellipse_function(
    lateral_contact_data = pd.DataFrame() , model = True, minP=None, maxP=None, minS=None, maxS=None
):
    
    if lateral_contact_data.empty:
        return model, minP, maxP, minS, maxS
    else:
        if minP == None:
            minP = lateral_contact_data["coord1"].min()
        if maxP == None:
            maxP = lateral_contact_data["coord1"].max()
        if minS == None:
            minS = lateral_contact_data["coord2"].abs().min()
        if maxS == None:
            maxS = lateral_contact_data["coord2"].max()

        a = (maxP - minP) / 2
        b = (maxS - minS) / 2

        po = minP + (maxP - minP) / 2

        p_locations = lateral_contact_data.loc[:, "coord1"].copy().to_numpy()

        s = np.zeros([len(p_locations), 2])

        s[np.logical_and(p_locations>minP, p_locations<maxP),0] =  b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 
        s[np.logical_and(p_locations>minP, p_locations<maxP),1] =  -b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 

        return s

def bell_shape(othercontact_data = pd.DataFrame(), mean_growth=None, minP=None, maxP=None, minS=None, maxS=None, vertex=None): 
    
    if othercontact_data.empty:
        return mean_growth
    

    c1c2_locations = othercontact_data.loc[:,['coord1','coord2']].to_numpy()
    
    midP = minP + (maxP-minP)/2
    midS = minS + (maxS-minS)/2
    
    mean = midP
    std = (maxP-minP)/2
    max_c0 = -vertex[0]
    x = c1c2_locations[:,0]
    y_out =max_c0*1/(std * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * std**2))
    c0 = np.zeros([len(c1c2_locations),2])

    c0[:,0] = y_out
    c0[:,1] = y_out
    return c0

## Create the 3D geological model.

First, we build the host rock (conformable stratigraphy), and then we build the intrusion, whose geometry is controlled by the host rock geometry. 

In [ ]:
model = GeologicalModel(lower_extent,upper_extent)
model.nsteps = [30,30,30]
model.data = model_data

conformable_feature = model.create_and_add_foliation('stratigraphy', nelements = 3000, solver = 'lu', interpolatortype = 'FDI')

In [ ]:
viewer_data = LavaVuModelViewer(model, background='white')
viewer_data.add_isosurface(conformable_feature, nslices = 10, paint_with = conformable_feature)

viewer_data.add_points(intrusion_data.loc[:,['X','Y','Z']].to_numpy(), name = 'intrusion data', pointsize = 10)

viewer_data.rotate([-87.74, 11.981, 0.71])
viewer_data.interactive()

In [ ]:
# Create intrusion
intrusion_frame_parameters = {'type' : 'interpolated',
                              'contact' :'floor', # reference contact to buil coordinate 0 of the intrusion frame
                              'contact_anisotropies':[conformable_feature]} # host rock controlling the intrusion geometry

vertical_conceptual_model = bell_shape

print(datetime.now().isoformat(timespec='seconds'))   

Laccolith = model.create_and_add_intrusion('laccolith', intrusion_frame_name = 'laccolith_frame',
                                           intrusion_frame_parameters = intrusion_frame_parameters,
                                           intrusion_lateral_extent_model = ellipse_function,
                                           intrusion_vertical_extent_model = vertical_conceptual_model,
                                          interpolatortype = 'FDI'
                                          )

print(datetime.now().isoformat(timespec='seconds'))  

## 3. Visualisation of intrusion frame

In [ ]:
intrusion_frame = Laccolith.intrusion_frame

viewer_frame = LavaVuModelViewer(model, background='white')

viewer_frame.add_isosurface(intrusion_frame[0], isovalue =0, colour = 'darkred')
viewer_frame.add_isosurface(intrusion_frame[1], isovalue =0, colour = 'darkgreen')
viewer_frame.add_isosurface(intrusion_frame[2], isovalue = 0, colour = 'blue')

viewer_frame.add_points(intrusion_data.loc[:,['X','Y','Z']].to_numpy(), name = 'intrusion data', pointsize = 10)
viewer_frame.add_points(np.array(lower_extent), name = 'low', pointsize = 1)
viewer_frame.add_points(np.array(upper_extent), name = 'upp', pointsize = 1)

# viewer_frame.add_data(intrusion_frame[0])
viewer_frame.add_data(intrusion_frame[1])
viewer_frame.add_data(intrusion_frame[2])
viewer_frame.rotate([-87.74, 11.981, 0.71])
viewer_frame.interactive()

## 4. Visualization of 3D Geological Model

In [ ]:
nn = 30 # improve model resolution by changing this value
model.nsteps = [nn,nn,nn]

In [ ]:
datetime.now().isoformat(timespec='seconds')   

In [ ]:
viewer = LavaVuModelViewer(model, background='white')

# add host rock
# viewer.add_isosurface(conformable_feature, nslices = 10, paint_with = conformable_feature)

# add intrusion data and intrusion contact isosurface
viewer.add_points(intrusion_data.loc[:,['X','Y','Z']].to_numpy(), name = 'intrusion data', pointsize = 15)
viewer.add_isosurface(Laccolith, isovalue = 0, colour = 'orchid')

# viewer.add_section(axis = 'y', value = 0.5, paint_with = Laccolith,  name = 'section_y')
# viewer.add_section(axis = 'x', value = 0.5, paint_with = Laccolith,  name = 'section_x')

viewer.add_points(np.array(lower_extent), name = 'low', pointsize = 1)
viewer.add_points(np.array(upper_extent), name = 'upp', pointsize = 1)

viewer.xmin = 0 
viewer.xmax = 1 
viewer.ymin = 0 
viewer.ymax = 1 
viewer.interactive()

In [ ]:
datetime.now().isoformat(timespec='seconds')   

## 5. Post-intrusion stratigraphy

In this example, we use the geometry of the intrusion to modified the host rock, so it captures the geometry after the emplacement of the laccolith. 
The intrusion frame and the contact points are used to compute the vertical and lateral variation of the intrusion contact, along the frame axes. This gives us a characterization of the intrusion thickness along the coordinate 1 and 2. We can use this to modified the scalar field of the host rock, so if shows the uplifting of the host rock generated by the growth of the laccolith. 


In [ ]:
from LoopStructural.modelling.features import BaseFeature

class PostIntrusionStratigraphy(BaseFeature):
    def __init__(
        self, name="post-intrusion stratigraphy", gradient_function=None, model=None,
        intrusion = None,
        pre_intrusion_stratigraphy = None,
        emplacement_mechanism = 'floor depression'
    ):
        BaseFeature.__init__(self, name = name)
        
        self.name = name
        self.gradient_function = gradient_function
        self.model = model
        self.pre_intrusion_stratigraphy = pre_intrusion_stratigraphy
        self.intrusion = intrusion
        self.emplacement_mechanism = emplacement_mechanism
        
    def min(self):

        if self.model is None:
            return 0
        return np.nanmin(self.evaluate_value(self.model.regular_grid((10, 10, 10))))

    def max(self):
        """
        Calculate average of the support values

        Returns
        -------
        max : double
            max value of the feature evaluated on a (10,10,10) grid in model area
        """
        if self.model is None:
            return 0
        return np.nanmax(self.evaluate_value(self.model.regular_grid((10, 10, 10))))

    def evaluate_value(self, xyz):
        v = np.zeros((xyz.shape[0]))
        
        pre_intrusion_stratigraphy_values = self.pre_intrusion_stratigraphy.evaluate_value(xyz)
        intrusion_points = self.intrusion.evaluate_value(xyz)
               
        intrusion_coord0_pts = self.intrusion.intrusion_frame[0].evaluate_value(xyz)
        intrusion_coord1_pts = self.intrusion.intrusion_frame[1].evaluate_value(xyz)
        intrusion_coord2_pts = self.intrusion.intrusion_frame[2].evaluate_value(xyz)
        
        thresholds, residuals, conceptual = self.intrusion.interpolate_lateral_thresholds(intrusion_coord1_pts)
        c2_minside_threshold = thresholds[0]
        c2_maxside_threshold = thresholds[1]
        
        thresholds, residuals, conceptual = self.intrusion.interpolate_vertical_thresholds(intrusion_coord1_pts, intrusion_coord2_pts)
        c0_minside_threshold = thresholds[1]
        c0_maxside_threshold = thresholds[0]
        
        intrusion_contact_points = self.intrusion.intrusion_frame.builder.intrusion_network_data.loc[:,['X','Y','Z']].to_numpy()
        intrusion_contact_points_vals = self.pre_intrusion_stratigraphy.evaluate_value(intrusion_contact_points)
        # print(intrusion_contact_points_vals)
        
        thickness = c0_maxside_threshold - c0_minside_threshold
        
        if self.emplacement_mechanism == 'floor depression':
            modified_stratigraphy = pre_intrusion_stratigraphy_values - thickness
        elif self.emplacement_mechanism == 'roof lifting':
            modified_stratigraphy = pre_intrusion_stratigraphy_values - thickness
            
        mask_inside_intrusion = (intrusion_coord0_pts>=c0_minside_threshold)*(intrusion_coord0_pts<=c0_maxside_threshold)*(intrusion_coord2_pts>=c2_minside_threshold)*(intrusion_coord2_pts<=c2_maxside_threshold)
        
        mask = np.logical_and(intrusion_coord0_pts>c0_minside_threshold,
                              np.logical_and(intrusion_coord2_pts>=(c2_minside_threshold-1),
                              intrusion_coord2_pts<=(c2_maxside_threshold+1)))
        
        # post_stratigraphy_values = modified_stratigraphy
        
        post_stratigraphy_values = pre_intrusion_stratigraphy_values
        
        post_stratigraphy_values[mask] = modified_stratigraphy[mask]
        post_stratigraphy_values[mask_inside_intrusion] = np.mean(intrusion_contact_points_vals)

        return post_stratigraphy_values #, c0_maxside_threshold, c0_minside_threshold
    
    def evaluate_value_original(self, xyz):
        v = np.zeros((xyz.shape[0]))
        
        pre_intrusion_stratigraphy_values = self.pre_intrusion_stratigraphy.evaluate_value(xyz)
        intrusion_points = self.intrusion.evaluate_value(xyz)
               
        intrusion_coord0_pts = self.intrusion.intrusion_frame[0].evaluate_value(xyz)
        intrusion_coord1_pts = self.intrusion.intrusion_frame[1].evaluate_value(xyz)
        intrusion_coord2_pts = self.intrusion.intrusion_frame[2].evaluate_value(xyz)
        
        thresholds, residuals, conceptual = self.intrusion.interpolate_lateral_thresholds(intrusion_coord1_pts)
        c2_minside_threshold = thresholds[0]
        c2_maxside_threshold = thresholds[1]
        
        thresholds, residuals, conceptual = self.intrusion.interpolate_vertical_thresholds(intrusion_coord1_pts, intrusion_coord2_pts)
        c0_minside_threshold = thresholds[1]
        c0_maxside_threshold = thresholds[0]

    def evaluate_gradient(self, xyz):
        v = np.zeros((xyz.shape[0], 3))
        if self.gradient_function is None:
            v[:, :] = np.nan
        else:
            v[:, :] = self.gradient_function(xyz)
        return v

    def min(self):
        return self._min

    def max(self):
        return self._max

In [ ]:
post_intrusion_strat = PostIntrusionStratigraphy(model=model,
                                                 intrusion = Laccolith,
                                                 pre_intrusion_stratigraphy = conformable_feature,
                                                 emplacement_mechanism = 'roof lifting')

In [ ]:
viewer_post_intrusion = LavaVuModelViewer(model, background='white')
# viewer_post_intrusion.nsteps = [60,60,60]

viewer_post_intrusion.add_isosurface(post_intrusion_strat, slices = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6], paint_with =post_intrusion_strat)
viewer_post_intrusion.add_isosurface(conformable_feature, slices = [-0.1], paint_with =conformable_feature)
viewer_post_intrusion.add_isosurface(Laccolith, isovalue = 0, colour = 'orchid')
viewer_post_intrusion.add_section(x = 0.5, paint_with = post_intrusion_strat, name = 'post int strat')
viewer_post_intrusion.add_section(x = 0.49, paint_with = conformable_feature, name = 'pre int strat')
# viewer_post_intrusion.add_scalar_field(Laccolith)
viewer_post_intrusion.ymin=0
viewer_post_intrusion.ymax=1
viewer_post_intrusion.xmin=0
viewer_post_intrusion.xmax=1
viewer_post_intrusion.rotate([0,90,90])
viewer_post_intrusion.interactive()

## 4. Visualisation of intrusion frame

In [ ]:
viewer2 = LavaVuModelViewer(model, background='white')

viewer2.add_isosurface(intrusion_frame[0], isovalue =0, colour = 'darkred')
viewer2.add_isosurface(intrusion_frame[1], isovalue =0, colour = 'darkgreen')
viewer2.add_isosurface(intrusion_frame[2], isovalue = 0, colour = 'blue')
viewer2.add_data(intrusion_frame[0])
viewer2.rotate([0,90,90])
viewer2.interactive()
viewer2.interactive()